---
title: Análise dos Lotes - Lotes para Uso
author: Geraldo Braz Junior
description: Identificar lotes que poderiam ser usados para habitações e/ou para equipamentos públicos.
params:
    ocupacao_btn:
        label: Ocupação
        input: select
        value: 'Edificação sem uso'
        choices: ['Edificação Subutilizada', 'Estacionamento', 'Edificação sem uso', 'Terreno não edificado']
        multi: False
    conservacao_btn:
        label: Conservação
        input: select
        value: 'Péssimo ou em Risco'
        choices: ['Regular', 'Ótimo', 'Bom', 'Péssimo ou em Risco', 'Perda']
        multi: False
    area_btn:
        label: Área
        input: slider
        value: 200
        min: 20
        max: 500
    equipamento_btn:
        label: Equipamento
        input: select
        value: 'Educação'
        choices: ['Assistência Social', 'Educação', 'Equipamento Cultural', 'Saúde', 'Segurança']
        multi: False
    tipo_btn:
        label: Shape dos Lotes
        input: select
        value: 'Polígono'
        choices: ['Polígono', 'Ponto', 'Calor']
        multi: False    
---

### TODO
- Identificar lotes elegíveis para melhoria habitacional.
    - Variáveis de Lotes: ocupação, tamanho (<80m2)
    - cruzar com dados de setores censitários (domicilio01, v018-v022 - esgotamento)
    - cruzar com dados de Zona de Interesse Social
    - cruzar com dados de densidade populacional


In [ ]:
import os
import sys

#path para a biblioteca do apiModulo. Ajuste de acordo com a necessidade
sys.path.insert(0, os.path.abspath('../'))
from apiModulo.api import *

## Obtendo os lotes da base

In [ ]:
lotes = obterCamada('lotes')

# Analisando ocupação na base
- Campo "finais_status_ocu"
- Ao todo 2640 lotes classificados dessa maneira

In [ ]:
ocupacao = lotes[~lotes['ocupacao'].isnull()]
print(ocupacao.shape[0])

#adicionando equipamentos informados pela prefeitura
equipamentos = obterCamada('equipamento_publico', simples=False)

In [ ]:
#estado inicial das variáveis
ocupacao_btn = 'Edificação sem uso'
conservacao_btn = 'Péssimo ou em Risco'
area_btn = 200
equipamento_btn = 'Educação'
tipo_btn = 'Polígono'

In [ ]:
#ação quando selecionar itens
filtrado = ocupacao[(ocupacao['AREA_HTL']>=area_btn*0.8) & (ocupacao['AREA_HTL']>=area_btn*1.2)  & (ocupacao['ocupacao']==ocupacao_btn)]
equip_filtrado = equipamentos[ (equipamentos['tipo'] == equipamento_btn) ]
print(f"Total selecionado: {filtrado.shape[0]}")

        
m = visMultiMapa(width='90%', MAPA_ZOOM=15)
if tipo_btn == 'Polígono':
    m = visMultiMapa(m, tipo='calor', dado=equip_filtrado, variavel='tipo')        
    m = visMultiMapa(m, tipo='choro', 
                dado=filtrado,                         
                variavel='ocupacao',
                alias=f"Lotes com  Ocupação: {ocupacao_btn}, áre mínima: {area_btn} e estado de conservação: {conservacao_btn}" )   
else: 
    como_pontos = filtrado
    como_pontos['geometria'] = como_pontos['geometria'].centroid
    if tipo_btn == 'Calor':
        m = visMultiMapa(m, tipo='marcador', dado=equip_filtrado, variavel='tipo')        
        m = visMultiMapa(m, tipo='calor', 
                    dado=como_pontos,  
                    variavel='ocupacao',
                    alias=f"Lotes com  Ocupação: {ocupacao_btn}, áre mínima: {area_btn} e estado de conservação: {conservacao_btn}" )   
    else:
        m = visMultiMapa(m, tipo='calor', dado=equip_filtrado, variavel='tipo')        
        m = visMultiMapa(m, tipo='marcador', 
                    dado=como_pontos,  
                    variavel='ocupacao',
                    alias=f"Lotes com  Ocupação: {ocupacao_btn}, áre mínima: {area_btn} e estado de conservação: {conservacao_btn}" )   
m